In [ ]:
from flask import Flask, request, render_template, redirect, url_for, session, Blueprint
import mysql.connector

app = Flask(__name__)
app.secret_key = 'your password'

# 創建 Blueprint 實例
delete_bp = Blueprint('delete_bp', __name__)

# 配置資料庫
db_config = {
    'user': 'root',
    'password': 'your password',
    'host': 'localhost',
    'database': 'sakila(or your database)'
}

# 添加 Blueprint 路由
@delete_bp.route('/delete/<int:address_id>', methods=['POST'])
def delete_address(address_id):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # 刪除指定的地址
    delete_query = "DELETE FROM address WHERE address_id = %s"
    cursor.execute(delete_query, (address_id,))
    conn.commit()

    cursor.close()
    conn.close()

    return redirect(url_for('index'))

# 註冊 Blueprint
app.register_blueprint(delete_bp)

@app.route('/', methods=['GET', 'POST'])
def index():
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # 初始化或從 session 中獲取上次看到的時間戳
    last_seen_timestamp = session.get('last_seen_timestamp', '1970-01-01 00:00:00')

    if request.method == 'POST':
        # 獲取表單輸入
        address = request.form['address']
        address2 = request.form['address2']
        district = request.form['district']
        city_id = request.form['city_id']
        postal_code = request.form['postal_code']
        phone = request.form['phone']
        location = request.form.get('location', None)

        # 將新記錄插入到資料庫中
        insert_query = """
            INSERT INTO address (address, address2, district, city_id, postal_code, phone, location, last_update)
            VALUES (%s, %s, %s, %s, %s, %s, ST_GeomFromText(%s), NOW())
        """
        cursor.execute(insert_query, (address, address2, district, city_id, postal_code, phone, 'POINT(0 0)' if not location else location))
        conn.commit()

        # 更新 session 中的時間戳（上次更新）
        session['last_seen_timestamp'] = last_seen_timestamp = cursor.lastrowid

    # 查詢自上次看到的時間戳以來新添加的記錄
    select_query = """
        SELECT address_id, address, address2, district, city_id, postal_code, phone, last_update
        FROM address
        WHERE last_update > %s
        ORDER BY last_update DESC
    """
    cursor.execute(select_query, (last_seen_timestamp,))
    new_addresses = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('index2.html', addresses=new_addresses)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.18.34.158:8080
Press CTRL+C to quit
172.18.34.158 - - [30/Sep/2024 09:46:41] "GET / HTTP/1.1" 200 -
172.18.34.158 - - [30/Sep/2024 09:46:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Sep/2024 09:48:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 09:48:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Sep/2024 09:49:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 09:49:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Sep/2024 09:49:44] "GET / HTTP/1.1" 200 -
172.18.34.158 - - [30/Sep/2024 09:51:28] "GET / HTTP/1.1" 200 -
172.18.34.158 - - [30/Sep/2024 09:53:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 10:01:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 10:01:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Sep/2024 10:02:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2024 10:03:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/20